# Named Entity Recognition (NER)
> Used NER to identify and redact sensitive information such as names, emails, phone numbers, and SSNs from customer comments—supporting privacy compliance and secure data handling

In [1]:
import spacy
import re
import pandas as pd

nlp = spacy.load("en_core_web_trf")

/Users/saranshbogati/Desktop/Projects/AI/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def redact_text(text):
    doc = nlp(text)

    redacted = text
    for ent in doc.ents:
        if ent.label_ in ["PERSON", "GPE", "ORG"]:
            redacted = redacted.replace(ent.text, "[REDACTED]")

    redacted = re.sub(
        r"\b\d{3}[-.\s]??\d{2}[-.\s]??\d{4}\b", "[REDACTED]", redacted
    )  # SSN
    redacted = re.sub(r"\b\d{10,}\b", "[REDACTED]", redacted)
    redacted = re.sub(r"\S+@\S+\.\S+", "[REDACTED]", redacted)

    return redacted

In [3]:
df = pd.read_csv("../data/hospital.csv")

df["Redacted_Feedback"] = df["Feedback"].apply(redact_text)


for i, text in enumerate(df["Feedback"].dropna()):
    doc = nlp(text)
    print(f"\nFeedback {i+1}: {text}")
    for ent in doc.ents:
        print(f"  - {ent.text} ({ent.label_})")
    if i == 50:
        break

print(df[["Feedback", "Redacted_Feedback"]].head(50))


Feedback 1: Good and clean hospital. There is great team of doctors and good OT facility. The medical facilities are all great with good in-patient post operative care by doctors, nurses. The housekeeping staff is also good but they can respond to calls from the room a little faster. 

Feedback 2: Had a really bad experience during discharge. They need to be sensitive and more transparent towards the patient and his/her family. 

Feedback 3: I have visited to take my second dose and Process was really smooth. Hospitality from all staffs are really appreciable. Shanti tooks good care and provide all details before vaccination. Thank you.
  - second (ORDINAL)
  - Process (PRODUCT)
  - Shanti (ORG)

Feedback 4:  That person was slightly clueless and offered only one package. But once i got to the hospital, I found a better and more economical one and decided to go ahead
  - only one (CARDINAL)

Feedback 5: There is great team of doctors and good OT facility.

Feedback 6: My primary conce